<a href="https://colab.research.google.com/github/ssawant/TensorFlow-in-Practice/blob/main/C2W4_Multiclass_Dropouts_RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

--2020-12-23 04:46:34--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 172.253.117.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   102MB/s    in 1.9s    

2020-12-23 04:46:37 (102 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

--2020-12-23 04:46:37--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/tmp/r

In [3]:
import os,zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print(f'rock: {len(os.listdir(rock_dir))}, paper: {len(os.listdir(paper_dir))}, scissors: {len(os.listdir(scissors_dir))}')

rock: 840, paper: 840, scissors: 840


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = '/tmp/rps'
VALIDATION_DIR = '/tmp/rps-test-set'

train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255.
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR, 
    target_size=(150,150),
    class_mode = 'categorical',
    batch_size=126
    )

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR, 
    target_size=(150,150),
    class_mode = 'categorical',
    batch_size=126
    )

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(150,150, 3)),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

model.summary()


Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_______________________________________

In [ ]:
model.compile(optimizer=tf.optimizers.RMSprop(lr=1e-04), loss=tf.losses.categorical_crossentropy, metrics=['accuracy'])
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

Epoch 1/25
20/20 [==============================] - 27s 967ms/step - loss: 1.0909 - accuracy: 0.3742 - val_loss: 1.0068 - val_accuracy: 0.6129
Epoch 2/25
20/20 [==============================] - 19s 961ms/step - loss: 1.0127 - accuracy: 0.5130 - val_loss: 0.8919 - val_accuracy: 0.7312
Epoch 3/25
20/20 [==============================] - 19s 954ms/step - loss: 0.9320 - accuracy: 0.5602 - val_loss: 0.7649 - val_accuracy: 0.5565
Epoch 4/25
20/20 [==============================] - 19s 948ms/step - loss: 0.8752 - accuracy: 0.6035 - val_loss: 0.6031 - val_accuracy: 0.8065
Epoch 5/25
20/20 [==============================] - 19s 949ms/step - loss: 0.7391 - accuracy: 0.6884 - val_loss: 0.6548 - val_accuracy: 0.6183
Epoch 6/25
20/20 [==============================] - 19s 954ms/step - loss: 0.6948 - accuracy: 0.7037 - val_loss: 0.4795 - val_accuracy: 0.8844
Epoch 7/25
20/20 [==============================] - 19s 944ms/step - loss: 0.6557 - accuracy: 0.7159 - val_loss: 0.4081 - val_accuracy: 0.8226

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.hisory['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()